# CADS API: constraints tests

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import xarray as xr

import cads_api_client
import cads_processing_api_service

In [3]:
api_url = os.getenv("CADS_API_URL", "http://cds2-dev.copernicus-climate.eu/api")
api_url

'http://cds2-dev.copernicus-climate.eu/api'

## Client instantiation

In [4]:
client = cads_api_client.ApiClient(url=api_url, key="00112233-4455-6677-c899-aabbccddeeff")
client

ApiClient(key='00112233-4455-6677-c899-aabbccddeeff', url='http://cds2-dev.copernicus-climate.eu/api')

## Check Collections ids

In [5]:
client.collections().collection_ids()

['derived-near-surface-meteorological-variables',
 'reanalysis-era5-land-monthly-means',
 'reanalysis-era5-land',
 'cams-global-reanalysis-eac4-monthly',
 'reanalysis-era5-single-levels',
 'reanalysis-era5-pressure-levels',
 'cams-global-reanalysis-eac4',
 'satellite-surface-radiation-budget']

## Constraints API validation

**Objective**: verify the correct functioning of the constraints API.

**Expected result:** client.valid_values() returns values that can be downloaded together with the input.

### Test 1 - Data availability

#### 1.0 - Get all available values

Consider the `"reanalysis-era5-single-levels"` dataset 

In order to get all available values we can leave the `request` param empty  

In [9]:
client.valid_values("reanalysis-era5-single-levels", {})

HTTPError: 500 Server Error: Internal Server Error for url: http://cds2-dev.copernicus-climate.eu/api/retrieve/v1/processes/reanalysis-era5-single-levels/constraints


#### 1.1 - Availability limits
The `"reanalysis-era5-single-levels"` dataset stops the 31th of october. Checking which values are valid for the year `"2022"` should return month numbers up to 8 (august).


In [8]:
response = client.valid_values("reanalysis-era5-single-levels", {"year" : ["2022"]})
response["month"]

HTTPError: 500 Server Error: Internal Server Error for url: http://cds2-dev.copernicus-climate.eu/api/retrieve/v1/processes/reanalysis-era5-single-levels/constraints

In [ ]:
print(response["year"])

#### 1.2  - Unavailable data

For a valid key, if none of the requested value(s) are valid, the API will return no valid combinations and suggest a list of valid values for the same key.


In [ ]:
print(client.valid_values("reanalysis-era5-single-levels", {"year":["1000"]}))

### Test 2 - Restricted params

Consider the `"cams-global-reanalysis-eac4-monthly"` dataset. 


In [ ]:
print(client.valid_values("cams-global-reanalysis-eac4-monthly", {}))

selecting `"product_type":"monthly_mean"` should not be compatible with any `"time"` value.


In [ ]:
response = client.valid_values("cams-global-reanalysis-eac4-monthly", {"product_type": ["monthly_mean"]})
response["time"]

On the countrary, if `"product_type"` is `"monthly_mean_by_hour_of_day"`, all `"time"` values should be available

In [ ]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {'product_type': ['monthly_mean_by_hour_of_day']})["time"]

the reverse is also true:

In [ ]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"time": ['00:00', '03:00', '18:00', '09:00', '15:00', '21:00', '06:00', '12:00']})["product_type"]

### Test 3 - invalid combination
Consider the `"cams-global-reanalysis-eac4-monthly"` dataset. 

There should be no valid values for `"time":"12:00"` and `"product type":"montly_mean"`.


In [ ]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"product_type" : ["montly_mean"], "time":["12:00"]})

The API responce suggests that adding `"monthly_mean_by_hour_of_day"` to the request may produce some valid values. 

Indeed:

In [ ]:
client.valid_values(
    "cams-global-reanalysis-eac4-monthly", 
    {"product_type" : ["montly_mean", "monthly_mean_by_hour_of_day"] , "time":"12:00"}
)

### Test 4 - Always valid params

Some params are always valid, meaning they can go along with any other combination. Theese are always returned by the API:

In [ ]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"product_type" : ["X"], "variable":["X"]})

### Test 5 - Invalid Keys


If some param with invalid or misspelled name is requested the API throws an error 

In [ ]:
client.valid_values("cams-global-reanalysis-eac4-monthly", {"invalid_param":["X"]})